In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
import xml.etree.ElementTree as ET
import pandas as pd
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def form_getter(file_list, path):
    master_list = []

    for i in tqdm(file_list):
        root = ET.parse(i).getroot()
        track = root[0]
        date = root[1]
        for race in root.iter("Race"):
            print(race)
            ids = []
            boxes = []
            weights = []
            dog_name = []
            for dog in race.iter("Dog"):
                if dog[0].text not in "12345678":
                    #print("longscratch")
                    continue
                else:
                    pass
                    #print(dog[0].text)


                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD") or ("=" in dog[0].text):
                    # print("scratched check working")
                    continue
                else:
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[0].text)
                    weights.append(dog[1].text)
                    dog_name.append(dog[2].text)


            zipall = list(
                zip(
                    ids,
                    boxes,
                    weights,
                    dog_name
                )
            )
            # print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (
                    race.attrib["id"],
                    date.text,
                    track.text,
                    race[3].text,
                    race[4].text,
                    race[5].text,
                ) + zipall[i]
                master_list.append(zipall[i])

    return master_list


In [3]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [4]:
#pickle Load results
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
form_file = open( 'results-df-merged-prices.npy', 'rb')
# form_file = open( 'results-df.npy', 'rb')
form = pickle.load(form_file)
form['datetime'] = pd.to_datetime(form['date'])
form

,@id_x,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,RaceId,@id_y,...,PPMIN,IPMAX,IPMIN,MORNINGTRADEDVOL,PPTRADEDVOL,IPTRADEDVOL,<!DOCTYPE html>,dog,loc,datetime
171235,849837292,8,SOUTHERN SHEETMETAL,08:16PM,545m,Mixed 4/5,Ballarat,01 Dec 22,849837292,558491864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01
1000615,846620229,8,GREYHOUNDS AS PETS STAKE,09:04PM,530m,Grade 5,Angle Park,01 Dec 22,846620229,592709559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01
1000604,846620228,7,@THEDOGSSA MIXED STAKE,08:40PM,530m,Mixed 4/5,Angle Park,01 Dec 22,846620228,529306503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01
1000605,846620228,7,@THEDOGSSA MIXED STAKE,08:40PM,530m,Mixed 4/5,Angle Park,01 Dec 22,846620228,520620677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01
1000606,846620228,7,@THEDOGSSA MIXED STAKE,08:40PM,530m,Mixed 4/5,Angle Park,01 Dec 22,846620228,570629991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112771,500543144,5,GREYHOUNDS NORTH MAIDEN DISTANCE,02:28PM,603m,Maiden Distance,Auckland (NZ),01 Dec 19,500543144,212745230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01
112770,500543144,5,GREYHOUNDS NORTH MAIDEN DISTANCE,02:28PM,603m,Maiden Distance,Auckland (NZ),01 Dec 19,500543144,184228722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01
112769,500543144,5,GREYHOUNDS NORTH MAIDEN DISTANCE,02:28PM,603m,Maiden Distance,Auckland (NZ),01 Dec 19,500543144,286157215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01
109309,500543149,10,MT WELLINGTON TAB STAKES,03:55PM,527m,Mixed 3/4,Auckland (NZ),01 Dec 19,500543149,236752141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01


In [58]:
tracks = form.track.unique().tolist()

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(form[['track']])
#enc = OneHotEncoder( categories=tracks)


In [ ]:
#Loading race results database
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms')
file_list = os.listdir()
path = os.getcwd()


file_list_new = [x for x in file_list if ".xml" in x]
file_list_new
#file_list = [r"forms\bf-BEN12Oct22.xml"]
fg = form_getter(file_list_new, path="")
fg

In [82]:
pred_df = pd.DataFrame(fg, columns=["race_id","date", "track", "datetime", "dist",'grade', "dog_id", "box", "extra1", "dogname"])
pred_df["dist"] = pred_df["dist"].astype(str).str[:-1].astype(float)
pred_df["box"] = pred_df["box"].astype(int)
pred_df = pred_df.sort_values('datetime')
pred_df

,race_id,date,track,datetime,dist,grade,dog_id,box,extra1,dogname
115,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,580425042,6,32466,SOLAR LOCK
112,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,557391059,3,42241,STORM SHELTER
116,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,580416015,7,64837,MASTER LOGAN
117,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,571666553,8,77364,INGA LEAH
114,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,566432220,5,74331,PRINCE OF MELTON
...,...,...,...,...,...,...,...,...,...,...
104,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,570895031,3,68,REBEL'S SPARK
105,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,666672184,4,4,CORPORATE BOX
106,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,580416021,5,76544,FIRE THE PAGE
107,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,664609569,6,T,PYRENEES BLAZER


In [83]:
pred_df.columns

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit(form[['track']])

transformed.transform(pred_df[['track']])

pred_df["tracksOnehot"] =  transformed.transform(pred_df[['track']]).tolist()
pred_df["tracksOnehot"]

115    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
112    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
116    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
117    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
114    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                           ...                      
104    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
105    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
106    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
107    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
108    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: tracksOnehot, Length: 363, dtype: object

In [84]:
pred_df

,race_id,date,track,datetime,dist,grade,dog_id,box,extra1,dogname,tracksOnehot
115,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,580425042,6,32466,SOLAR LOCK,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
112,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,557391059,3,42241,STORM SHELTER,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
116,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,580416015,7,64837,MASTER LOGAN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
117,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,571666553,8,77364,INGA LEAH,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
114,839681531,03 Nov 22,Horsham,03 Nov 22 01:08AM,410.0,Grade 7,566432220,5,74331,PRINCE OF MELTON,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
104,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,570895031,3,68,REBEL'S SPARK,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
105,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,666672184,4,4,CORPORATE BOX,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
106,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,580416021,5,76544,FIRE THE PAGE,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
107,839681528,03 Nov 22,Horsham,03 Nov 22 12:49AM,410.0,Maiden,664609569,6,T,PYRENEES BLAZER,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
race_forms = form.groupby(["race_id"])
dog_forms = form.groupby(["dog_id"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details = []
dog_stats_df = []
race_pred_forms = pred_df.groupby(["race_id"], sort=False)

# for i, j in tqdm(race_forms):
#     print( f"{i=},{j=}")
#     break


for i, j in tqdm(race_pred_forms):

    race_date = pd.to_datetime(j["date"].iloc[0])
    dogs = j.dog_id
    race_id = i
    race_grade = j.grade.iloc[1]
    race_dist = j.dist.iloc[1]
    trackOHE = j.tracksOnehot.iloc[1]
    track_name = j.track.iloc[1]
    dog_info = []
    num_features = 16
    

    #Starts off with blank outfits of what we need to fill, are then filled using box pos as indexer
    blank_stats = [-1]*num_features
    blank_race = [blank_stats]*8
    blank_classes = [8]*8
    blank_margins = [80]*8
    blank_sp = [0]*8
    blank_bsp = [0]*8
    blank_dogname = ["blank"]*8
    if len(dogs)>8:
        continue
    i = 0
    for dog in dogs:
        try:
            #print(f"dog id = {dog}")
            prev_form = dog_forms.get_group(dog)
            prev_form = prev_form[(prev_form["datetime"] < race_date)]
            #print(prev_form)
            previous_race = prev_form.race_id.iloc[0]
            prev_races = prev_form.shape[0]
            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            if prev_races:
                dog_margin_last = prev_form.margin.iloc[0]
                dog_speed_last = prev_form.speed.iloc[0]
            else:
                dog_margin_last = -1
                dog_speed_last = -1
            # print(dog_margin_last)
            dog_race_count = prev_form.shape[0]
            dog_wins = prev_form[prev_form["place"] == 1].shape[0]
            dog_places = prev_form[prev_form["place"] <= 3].shape[0]
            dog_box = j.box.iloc[i]
            dog_weight = prev_form.weigth.iloc[-1] # weight not in forms for prediction going to use last weight
            stats = [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                    ]
            #stats = [-1 if math.isnan(x) else x for x in stats]
            dog_stats = (
                dog,
                stats,
                previous_race
            )

            #Fill in the blank outputs
            idx = dog_box-1

        except Exception as e:
            stats = [-1]*16
            previous_race = -1
            print(e)

        blank_race[idx] = stats
        blank_dogname[idx] = j.dogname.iloc[i]


        dog_info.append(dog_stats)
        dog_stats_df.append([dog,
                j.dogname.iloc[i],
                race_id,
                race_grade,
                race_date,
                trackOHE,
                track_name,
                race_dist,
                stats,
                previous_race])
        #print(f"{dog_stats_df=}")

        i = i + 1

    
    

    dog_info.sort(key=itemgetter(0))
    # dog_stats_df.append(dog_info)
    input_layer = (
        trackOHE
        + [race_dist]
        + [item for sublist in blank_race for item in sublist]
    )
    
    # if len(dogs) > 8:
    #     print(f"{race_id=}")
    #     print(f"{race_date=}")
    #     print(f"{j.track_x.iloc[1]=}")
    classes = blank_classes
    prices = blank_sp
    margins = blank_margins
    betfairSP = blank_bsp
    dogname = blank_dogname
    race_input_layer.append(input_layer)
    race_classes.append(classes)
    full_details.append(
        (race_id, input_layer, classes, prices,  margins, betfairSP, dogname)
    )

        

    # n += 1
    # if n >10:
    #     break




In [98]:
dfx = pd.DataFrame(dog_stats_df, columns= ['dogid','dog_name','raceid','race_grade','date','trackOHE','track_name','dist','stats', 'prev_race'])
dfx

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,prev_race
0,580425042,SOLAR LOCK,839681531,Grade 7,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[6, 0.05833943089430895, 0.0008445480486331037...",838466586
1,557391059,STORM SHELTER,839681531,Grade 7,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[3, 0.05926484528543352, 0.0007078990228496344...",833510212
2,580416015,MASTER LOGAN,839681531,Grade 7,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[7, 0.05838486901535683, 0.0009964115709727268...",789350050
3,571666553,INGA LEAH,839681531,Grade 7,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[8, 0.05863304014304284, 0.0006931339215671378...",835824832
4,566432220,PRINCE OF MELTON,839681531,Grade 7,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[5, 0.058770486656200945, 0.000628896797296129...",832457597
...,...,...,...,...,...,...,...,...,...,...
358,570895031,REBEL'S SPARK,839681528,Maiden,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[3, 0.059902912621359224, nan, 0.0599029126213...",828072346
359,666672184,CORPORATE BOX,839681528,Maiden,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[4, 0.058512195121951216, nan, 0.0585121951219...",837081330
360,580416021,FIRE THE PAGE,839681528,Maiden,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[5, 0.05963834539444295, 0.0005955354087876264...",837081331
361,664609569,PYRENEES BLAZER,839681528,Maiden,2022-11-03,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Horsham,410.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",-1


In [97]:
with open("prediction_inputs_gru.npy", "wb") as fp:   #Pickling
    
    pickle.dump(dfx, fp)

In [11]:
test_df = pd.DataFrame(race_input_layer)